In [2]:
import os
from PIL import Image
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
from utils import datasets
from utils import utils
from utils import train_val
from utils import net #网络文件于此
import warnings

# 完全禁用警告
warnings.filterwarnings("ignore")

In [3]:
from utils.utils import Config,Logs,BestSelector
config=utils.Config(
    dataset_sep=[
        0.82,0.17,0.01          
        ],
    resize_size=224,
    batch_size=64,
    lr=0.001,
    epochs=40,
    hidden_size=256,
    optim="Adam",
    momentum=0.9,
    weight_decay=1e-4,
    seed=42,
    mean= [0.50638 ,0.49962538 ,0.45205265],
    std=[0.23568255 ,0.24141274 ,0.25167742],
    source_dir=r"Cifar-10",#原始数据集，每个分类一个文件夹，每个文件夹里包含多个图片
    data_path=r"data\Cifar-10",#项目数据集
    # classes=["Apple","Carambola","Pear","Plum","Tomatoes"],
    device="cuda" if torch.cuda.is_available() else "cpu",
)
print(config)

dataset_sep : [0.82, 0.17, 0.01] 
resize_size : 224 
batch_size : 64 
lr : 0.001 
epochs : 40 
hidden_size : 256 
optim : Adam 
momentum : 0.9 
weight_decay : 0.0001 
seed : 42 
mean : [0.50638, 0.49962538, 0.45205265] 
std : [0.23568255, 0.24141274, 0.25167742] 
source_dir : Cifar-10 
data_path : data\Cifar-10 
device : cuda 


In [4]:
from tqdm import tqdm

#设置随机数种子
torch.manual_seed(config.seed)

isSplit= (not datasets.check_data_exist(config.data_path))#数据集不存在，则从原始数据存放处，转移数据集
if isSplit:
    print(f"{config.data_path} 数据集不存在，将从source_dir:{config.source_dir}中获取数据")
    print(f"清理源文件夹:{config.data_path}")
    datasets.clear_folder(config.data_path)
    
    for dir in tqdm(config.classes,desc="处理原始数据："):
        source_dir=os.path.join(config.source_dir, dir)
        print(source_dir)
        datasets.split_data(source_dir,target_dir=config.data_path,label=dir,sep=config.dataset_sep)
else:
    print(f"{config.data_path} 数据集已存在，无需重新划分")



data\Cifar-10 数据集已存在，无需重新划分


In [5]:
'''
def get_data_path(typ,path=config.data_path):
    return os.path.join(path,typ)

train_dataset=datasets.CustomImageDataset(
    get_data_path("train"),
    classes=config.classes,
    transform=datasets.get_transform(
        resize_size=config.resize_size,
        mean=config.mean,std=config.std
        )
)

val_dataset=datasets.CustomImageDataset(
    get_data_path("val"),
    classes=config.classes,
    transform=datasets.get_transform(
        "val",
        config.resize_size,
        
        mean=config.mean,
        std=config.std
        )
)
'''


'\ndef get_data_path(typ,path=config.data_path):\n    return os.path.join(path,typ)\n\ntrain_dataset=datasets.CustomImageDataset(\n    get_data_path("train"),\n    classes=config.classes,\n    transform=datasets.get_transform(\n        resize_size=config.resize_size,\n        mean=config.mean,std=config.std\n        )\n)\n\nval_dataset=datasets.CustomImageDataset(\n    get_data_path("val"),\n    classes=config.classes,\n    transform=datasets.get_transform(\n        "val",\n        config.resize_size,\n        \n        mean=config.mean,\n        std=config.std\n        )\n)\n'

#### 加载数据集，可本地数据集。调用datasets.CustomImageDataset

In [6]:
#加载cifar-10
import torchvision
train_val_dataset = torchvision.datasets.CIFAR10(
    root='./data/Cifar-10', 
    train=True, 
    download=True, 
    transform=datasets.get_transform(
        chance="train",
        resize_size=config.resize_size,
        mean=(0.5, 0.5, 0.5), 
        std=(0.5, 0.5, 0.5)
    )
)
train_dataset,val_dataset=datasets.get_parts_of_datasets(train_val_dataset,rate=0.75,only_train=False)#训练验证3-1开

test_dataset = torchvision.datasets.CIFAR10(root='./data/Cifar-10', train=False, download=True, transform=datasets.get_transform(
    chance="val",
    resize_size=config.resize_size,
    mean=(0.5, 0.5, 0.5), 
    std=(0.5, 0.5, 0.5)
))
print(f"原始训练集大小：{len(train_dataset)}")
print(f"原始验证集大小：{len(val_dataset)}")
print(f"原始测试集大小：{len(test_dataset)}")

Files already downloaded and verified
Files already downloaded and verified
原始训练集大小：37500
原始验证集大小：12500
原始测试集大小：10000


#### load数据集，并根据数据量进行裁剪

In [7]:
crop_rate=0.25
train_dataset_crop=datasets.get_parts_of_datasets(train_dataset,crop_rate)
val_dataset_crop=datasets.get_parts_of_datasets(val_dataset,crop_rate)
test_dataset_crop=datasets.get_parts_of_datasets(test_dataset,crop_rate)
train_dataset_size=len(train_dataset_crop)
val_dataset_size=len(val_dataset_crop)
test_dataset_size=len(test_dataset_crop)

print(f"本次训练用训练集大小：{len(train_dataset_crop)}")
print(f"本次训练用验证集大小：{len(val_dataset_crop)}")
print(f"本次测试用测试集大小：{len(test_dataset_crop)}")

train_loader=DataLoader(train_dataset_crop,batch_size=config.batch_size,shuffle=True,drop_last=True)
val_loader=DataLoader(val_dataset_crop,batch_size=config.batch_size,shuffle=False)
test_loader=DataLoader(test_dataset_crop,batch_size=config.batch_size,shuffle=False)

print(f"tarin_dataloader加载完毕, {len(train_loader)}个batch, batch大小为{config.batch_size}")
print(f"val_dataloader  加载完毕, {len(val_loader)}个batch, batch大小为{config.batch_size}")
print(f"test_dataloader 加载完毕, {len(test_loader)}个batch, batch大小为{config.batch_size}")


本次训练用训练集大小：9375
本次训练用验证集大小：3125
本次测试用测试集大小：2500
tarin_dataloader加载完毕, 146个batch, batch大小为64
val_dataloader  加载完毕, 49个batch, batch大小为64
test_dataloader 加载完毕, 40个batch, batch大小为64


#### 记录本次训练和测试用的数据量

In [7]:
config.update(
    train_datasize=train_dataset_size,
    val_datasetsize=val_dataset_size,
    test_datasetsize=test_dataset_size,
    # epochs=10,
    classes=train_val_dataset.classes,#原始数据集保留classes
)
config

dataset_sep : [0.82, 0.17, 0.01] 
resize_size : 224 
batch_size : 64 
lr : 0.001 
epochs : 40 
hidden_size : 256 
optim : Adam 
momentum : 0.9 
weight_decay : 0.0001 
seed : 42 
mean : [0.50638, 0.49962538, 0.45205265] 
std : [0.23568255, 0.24141274, 0.25167742] 
source_dir : Cifar-10 
data_path : data\Cifar-10 
device : cuda 
train_datasize : 9375 
val_datasetsize : 3125 
test_datasetsize : 2500 
classes : ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 

#### 加载预训练模型

In [8]:

def get_pretrained(hidden_size=config.hidden_size,config=config):
    '''
        获取预训练模型
        @param hidden_size: 隐藏层大小
        @param config: 配置文件
        @return: 预训练模型
    '''
    model=net.BinaryClassificationMobileNetV3Large(out_size=len(config.classes))
    return model.to(config.device)



In [9]:
#查看数据加载情况
for inputs, labels in train_loader:
    inputs=inputs.to(config.device)
    labels=labels.to(config.device)
    print(inputs.shape)
    print(labels.shape)
    config.update(inputs_shape=inputs.shape)
    break
config

torch.Size([64, 3, 224, 224])
torch.Size([64])


dataset_sep : [0.82, 0.17, 0.01] 
resize_size : 224 
batch_size : 64 
lr : 0.001 
epochs : 40 
hidden_size : 256 
optim : Adam 
momentum : 0.9 
weight_decay : 0.0001 
seed : 42 
mean : [0.50638, 0.49962538, 0.45205265] 
std : [0.23568255, 0.24141274, 0.25167742] 
source_dir : Cifar-10 
data_path : data\Cifar-10 
device : cuda 
train_datasize : 9375 
val_datasetsize : 3125 
test_datasetsize : 2500 
classes : ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 
inputs_shape : torch.Size([64, 3, 224, 224]) 

In [10]:

bestMod=utils.BestSelector(acc=0)
train_logs=utils.Logs()
model=get_pretrained()
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCELoss()
if config.optim == "Adam":
    optimizer = optim.Adam(model.parameters(), lr=config.lr)
else:
    optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=0.9, weight_decay=1e-4)
    
bestMod,train_logs=train_val.train_model(
            model, 
            criterion,
            optimizer, 
            train_loader,
            val_loader,
            bestMod=bestMod,
            train_logs=train_logs,
            config=config, 
            # num_epochs=config.epochs
        )



Training Epochs:   2%|▎         | 1/40 [00:43<28:18, 43.54s/it]

Epoch 1/40, Loss: 1.9357,  Val Accuracy: 0.2509, AP: 0.1160, Precision: 0.2510, Recall: 0.2474
当前最好的模型： acc : 0.25088 ,loss : 1.9357111747950724 ,precision : 0.25099821661601257 ,recall : 0.24743728613203914 ,ap : 0.1160131640555689 ,epoch : 0 


Training Epochs:   5%|▌         | 2/40 [01:24<26:45, 42.26s/it]

Epoch 2/40, Loss: 1.8196,  Val Accuracy: 0.4550, AP: 0.0654, Precision: 0.5348, Recall: 0.4534
当前最好的模型： acc : 0.45504 ,loss : 1.8196199054587376 ,precision : 0.5348011596004018 ,recall : 0.45335697878004266 ,ap : 0.0653989959761442 ,epoch : 1 


Training Epochs:   8%|▊         | 3/40 [02:04<25:18, 41.04s/it]

...

Epoch 38/40, Loss: 1.6553,  Val Accuracy: 0.7043, AP: 0.0644, Precision: 0.7328, Recall: 0.7014
当前最好的模型： acc : 0.74336 ,loss : 1.6713340192624968 ,precision : 0.7507277672405344 ,recall : 0.7429182764754364 ,ap : 0.08467647449494926 ,epoch : 32 


Training Epochs:  98%|█████████▊| 39/40 [25:15<00:38, 38.53s/it]

Epoch 39/40, Loss: 1.6491,  Val Accuracy: 0.7318, AP: 0.0851, Precision: 0.7479, Recall: 0.7307
当前最好的模型： acc : 0.74336 ,loss : 1.6713340192624968 ,precision : 0.7507277672405344 ,recall : 0.7429182764754364 ,ap : 0.08467647449494926 ,epoch : 32 


Training Epochs: 100%|██████████| 40/40 [25:54<00:00, 38.86s/it]

Epoch 40/40, Loss: 1.6503,  Val Accuracy: 0.7053, AP: 0.0724, Precision: 0.7162, Recall: 0.7024
当前最好的模型： acc : 0.74336 ,loss : 1.6713340192624968 ,precision : 0.7507277672405344 ,recall : 0.7429182764754364 ,ap : 0.08467647449494926 ,epoch : 32 


In [11]:
print(f"{config.epochs} epoch中 最好的模型")
print(bestMod)

40 epoch中 最好的模型
acc : 0.74336 ,loss : 1.6713340192624968 ,precision : 0.7507277672405344 ,recall : 0.7429182764754364 ,ap : 0.08467647449494926 ,epoch : 32 


#### 保存模型超参数和训练日志

In [4]:
saveDir=r'save_weights'
saveDir=os.path.join(saveDir,f'{bestMod.model.__class__.__name__}-acc={round(bestMod.acc,5)}-loss={round(bestMod.loss,6)}-max_epochs={config.epochs}-1100'),
utils.saveProcess(
    saveDir=saveDir,
    bestMod=bestMod,
    train_log=train_logs,
    config=config
)

NameError: name 'bestMod' is not defined

#### 进行测试

In [10]:
import torch
from utils import utils,train_val
import os

# config=Config(os.path.join(dir,'config.json'))
# model=BestSelector(os.path.join(dir,'metrics.json'))
saveDir=r'save_weights\BinaryClassificationMobileNetV3Large-acc=0.74336-loss=1.671334-max_epochs=40-1100'

Model,config,logs=utils.loadProcess(saveDir=saveDir)
metrics=train_val.validate_model(
    model=Model.model,
    val_loader=test_loader,
    device=config.device,
    only_val=True
)

metrics

模型测试中:: 100%|██████████| 40/40 [00:04<00:00,  9.13it/s]


{'accuracy': 0.7892,
 'precision': 0.7911782819812854,
 'recall': 0.7895432141222342,
 'f1': 0.7868892957982461,
 'ap': 0.0726235784868686}

#### 保存数据到tensorboard

In [1]:
from utils import metrics,utils


recoder=metrics.TensorboardRecorder(
    log_dir="runs/"
)
recoder.logs_scalars(
    logs.logs,
    prefix="train"
)